In [19]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# import seaborn as sns
# import sklearn
import warnings

from sklearn.preprocessing import LabelEncoder
# from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.tree import plot_tree
 
warnings.filterwarnings('ignore')

# import torch
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
def Extract_time(timestamp):
       time = int(timestamp[11:13]) * 60 + int(timestamp[14:16])
       return time

In [21]:
# using = {1,2,3,4,5,6,7} # all
# using = {1,2,3,4} # shorties
# using = {5,6,7} # longers
using = {5,6} # custom

papers = ['ad', 'brabantsdagblad', 'destentor', 'nu', 'parool', 'trouw', 'volksrant']
features = ['brand', 'publication_timestamp', 'num_words', 'main_section', 'user_needs', 'lda_topics']
# features = ['brand', 'num_words']
target = 'relevance_window_in_hours'
df = None

with open('stop_words') as file:
    stop_words = file.read().split()
papers = [papers[x - 1] for x in using]

if 1 in using:
    df = pd.concat([df, pd.read_csv('dataset/uva-relevance-windows-ad.csv', sep=';')], ignore_index=True)
    # ad: has 99122 entries
    # target mean: 38.9

if 2 in using:
    df = pd.concat([df, pd.read_csv('dataset/uva-relevance-windows-brabantsdagblad.csv', sep=';')], ignore_index=True)
    # brabantsdagblad: has 34768 entries
    # target mean: 31.8

if 3 in using:
    df = pd.concat([df, pd.read_csv('dataset/uva-relevance-windows-destentor.csv', sep=';')], ignore_index=True)
    # destentor: has 36212 entries
    # target mean: 33.3
    
if 4 in using:
    df = pd.concat([df, pd.read_csv('dataset/uva-relevance-windows-nu.csv', sep=';')], ignore_index=True)
    # nu: has 16257 entries
    # target mean: 29.4

if 5 in using:
    df = pd.concat([df, pd.read_csv('dataset/uva-relevance-windows-parool.csv', sep=';')], ignore_index=True)
    # parool: has 7729 entries
    # target mean: 88.8

if 6 in using:
    df = pd.concat([df, pd.read_csv('dataset/uva-relevance-windows-trouw.csv', sep=';')], ignore_index=True)
    # trouw: has 8395 entries
    # target mean: 93.2

if 7 in using:
    df = pd.concat([df, pd.read_csv('dataset/uva-relevance-windows-volkskrant.csv', sep=';')], ignore_index=True)
    # volkskrant: has 10983 entries
    # target mean: 90.5

df.reset_index(inplace=True)
df.head()
df.info()
# df.value_counts('user_needs')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16124 entries, 0 to 16123
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   index                       16124 non-null  int64  
 1   article_id                  16124 non-null  object 
 2   brand                       16124 non-null  object 
 3   publication_timestamp       16124 non-null  object 
 4   publication_weekday         16124 non-null  object 
 5   title                       16124 non-null  object 
 6   first_paragraph             16122 non-null  object 
 7   full_body                   16122 non-null  object 
 8   num_words                   16124 non-null  int64  
 9   num_paragraphs              16124 non-null  int64  
 10  author_ids                  16124 non-null  object 
 11  labels                      15376 non-null  object 
 12  main_section                14175 non-null  object 
 13  subsections                 1 n

In [22]:
df = df.filter(features + [target])

if 'publication_timestamp' in features:
    df['publication_timestamp'] = df['publication_timestamp'].map(Extract_time)

if 'user_needs' in features:
    df['user_needs'] = df['user_needs'].map(eval)

    normalized_info = pd.json_normalize(df['user_needs'])
    df = pd.concat([df.drop(columns=['user_needs']), normalized_info], axis=1)

if 'lda_topics' in features:
    df.rename(columns={'brand':'Brand'}, inplace=True)
    df['lda_topics'] = df['lda_topics'].map(eval)

    normalized_info = pd.json_normalize(df['lda_topics'])
    df = pd.concat([df.drop(columns=['lda_topics']), normalized_info], axis=1)
    df.rename(columns={'brand':'lda_brand'}, inplace=True)
    df.rename(columns={'Brand':'brand'}, inplace=True)

df = pd.concat([df.drop(columns=[target]), df[target]], axis=1)

X = df.drop(columns=[target]) # features
y = df.iloc[:,-1].values      # Target variable

In [23]:
%%script false --no-raise-error # skip cell, needed for classifier

df[target] = pd.cut(df[target], [0, 3, 6, 12, 24, 48, 120, 1000], labels=['0-3', '3-6', '6-12','12-24','24-48','48-120','120-1000'])
X = df.drop(columns=[target])  # features
y = df.iloc[:,-1].values    # Target variable

In [24]:
label_encoder = LabelEncoder()
x_categorical = X.select_dtypes(include=['object']).apply(label_encoder.fit_transform)
x_numerical = X.select_dtypes(exclude=['object']).values
x = pd.concat([pd.DataFrame(x_numerical), x_categorical], axis=1).values

# Splitting the data
x_train, x_test, y_train, y_test = train_test_split( 
          x, y, test_size = 0.1)

In [25]:
regressor = GradientBoostingRegressor(
    n_estimators=50, 
    learning_rate=0.05,
    min_samples_leaf=25,
    max_depth=25,
    random_state=0,
)
# regressor = GradientBoostingClassifier(
#     loss='log_loss',
#     criterion='squared_error',
#     n_estimators=100, 
#     learning_rate=0.005,
#     min_samples_leaf=10,
#     random_state=0,
# )

# Fit the regressor with x and y data
regressor.fit(x_train, y_train)

GradientBoostingRegressor(learning_rate=0.05, max_depth=25, min_samples_leaf=25,
                          n_estimators=50, random_state=0)

In [26]:
# Evaluating the model
from sklearn.metrics import mean_squared_error, r2_score

# Making predictions
predictions = regressor.predict(x_test)

y_train_pred = regressor.predict(x_train)
y_test_pred = regressor.predict(x_test)

# Evaluating the model
# y_train_pred = [int(x) for x in y_train_pred] # needed for regressor
# y_test_pred = [int(x) for x in y_test_pred] # needed for regressor

# train_accuracy = accuracy_score(y_train, y_train_pred)
# test_accuracy = accuracy_score(y_test, y_test_pred)


# print("Train Accuracy: %.1f%%" %(train_accuracy * 100))
# print("Test Accuracy: %.1f%%" %(test_accuracy * 100))


mse = mean_squared_error(y_test, predictions) # needed for regressor
print(f'Mean Squared Error: {mse}') # needed for regressor

r2 = r2_score(y_test, predictions) # needed for regressor
print(f'R-squared: {r2}') # needed for regressor

# takes a long time to compute
# cross_value_score = cross_val_score(regressor, x_train, y_train)
# print(f'cross_value_score: {cross_value_score}')


Mean Squared Error: 7634.396365285173
R-squared: 0.2572883335450177


## 100 trees
### classifier
Train Accuracy: 50.2% <br>
Test Accuracy: 45.5% <br>

In [55]:
%%script false --no-raise-error # skip cell, Visualization

X_grid = np.linspace(0,1000,1)
X_grid = X_grid.reshape(len(X_grid),1) 
  
plt.scatter(X,y, color='blue') #plotting real points
plt.plot(X_grid, regressor.predict(X_grid),color='green') #plotting for predict points
  
plt.title("Random Forest Regression Results")
plt.xlabel('Position level')
plt.ylabel('Salary')
plt.show()

In [56]:
%%script false --no-raise-error # skip cell, Tree Visualization

# Plot the decision tree
def plotting_tree(tree):
    # print("Worker thread running")
    n, tree_to_plot = tree
    plt.figure(figsize=(20, 10))
    plot_tree(tree_to_plot, feature_names=df.columns.tolist(), filled=True, rounded=True, fontsize=10)
    plt.title(f"Boosting Tree {n}")
    plt.show()

In [53]:
%%script false --no-raise-error # skip cell, Tree Visualization

for tree in enumerate(regressor.estimators_):
    plotting_tree(tree)

In [54]:
%%script false --no-raise-error # skip cell, Tree Visualization

import concurrent.futures    

pool = concurrent.futures.ThreadPoolExecutor(max_workers=10)

for tree in enumerate(regressor.estimators_):
    pool.submit(plotting_tree, tree)

print('All trees submitted')

pool.shutdown(wait=True)

print("Main thread continuing to run")